In [1]:
import nltk
import torch
import matplotlib
import numpy as np
import pandas as pd
import torch.nn as nn
import hiddenlayer as hl
from functools import reduce
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/karthik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Load Data

In [2]:
# Load Dataset
data = pd.read_csv('Training Data.csv')

# Slice relevant colums
rel_data = data.loc[:,['Company Name','Business Description','Industry Classifications']]
print("# Unique Companies in training:", rel_data["Company Name"].nunique())

# Extract labels
rel_data['labels'] = rel_data["Industry Classifications"].str.split(";", expand=True).iloc[:,[0]]
trunc_training_data = rel_data.loc[:, ['Company Name','Business Description', 'labels']]
trunc_training_data['labels'] = trunc_training_data['labels'].str.replace("\(Primary\)", "")
trunc_training_data['labels'] = trunc_training_data['labels'].str.replace(" ", "")
trunc_training_data['labels'] = trunc_training_data['labels'].str.lower()

print("# unique labels in training", trunc_training_data["labels"].nunique())

# Unique Companies in training: 2002
# unique labels in training 197


In [4]:
# Load inference data
inference_data = pd.read_csv("Company_Business_Description_.csv")

# Slice relevant colums
inference_data = inference_data.loc[:,['Company Name','Business Description']]
print("# Unique Companies in Inference data:", inference_data["Company Name"].nunique())

# Unique Companies in Inference data: 8425


# Pre-processing

In [5]:
# Basic preprocessing
trunc_training_data['Business Description'] = trunc_training_data['Business Description'].str.replace(r'[0-9]', " ")
trunc_training_data['Business Description'] = trunc_training_data['Business Description'].str.replace(r'[^\w\s]', " ")
trunc_training_data['Business Description'] = trunc_training_data['Business Description'].str.replace(r"\bLLC\b | \bInc\b | \bst\b | \brd\b | \bST\b | \bnd\b", " ")

# Clean inference data
inference_data['Business Description'] = inference_data['Business Description'].str.replace(r'[0-9]', " ")
inference_data['Business Description'] = inference_data['Business Description'].str.replace(r'[^\w\s]', " ")
inference_data['Business Description'] = inference_data['Business Description'].str.replace(r"\bLLC\b | \bInc\b | \bst\b | \brd\b | \bST\b | \bnd\b", " ")

In [6]:
# Removing Stop words
stop = stopwords.words('english')
trunc_training_data['Business Description'] = trunc_training_data['Business Description'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

# Remove infrequent words
freq = pd.Series(' '.join(trunc_training_data['Business Description']).split()).value_counts()[-10:]
freq = list(freq.index)
trunc_training_data['Business Description'] = trunc_training_data['Business Description'].str.lower().apply(lambda x: " ".join(x for x in x.split() if x not in freq))
#trunc_training_data['Business Description'].head()
trunc_training_data = trunc_training_data[trunc_training_data['Business Description']!='']
print(len(trunc_training_data))

# Save the cleaned data
#trunc_training_data.to_csv("Cleaned_training_data.csv")

# Clean inference data
inference_data['Business Description'] = inference_data['Business Description'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
freq = pd.Series(' '.join(inference_data['Business Description']).split()).value_counts()[-10:]
freq = list(freq.index)
inference_data['Business Description'] = inference_data['Business Description'].str.lower().apply(lambda x: " ".join(x for x in x.split() if x not in freq))
inference_data.head()
#inference_data.to_csv('Cleaned_inference_data.csv')

1970


,Company Name,Business Description
0,"024 Pharma, Inc. (OTCPK:EEIG)",pharma provides healthcare products worldwide ...
1,"1-800-FLOWERS.COM, Inc. (NasdaqGS:FLWS)",flowers com together subsidiaries provides gou...
2,12 Retech Corporation (OTCPK:RETC),retech corporation subsidiaries operates integ...
3,"1347 Property Insurance Holdings, Inc. (Nasdaq...",property insurance holdings subsidiaries provi...
4,1847 Holdings LLC (OTCPK:EFSH),holdings subsidiaries provides range products ...


In [7]:
trunc_training_data['labels'].nunique()

196

In [8]:
# integer encode
values = np.array(trunc_training_data['labels'])
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded.min(), integer_encoded.max())

0 195


In [9]:
train = pd.DataFrame({"text": trunc_training_data.iloc[:, [1]].values.tolist(), 'lable_text':values, "label": integer_encoded})
train.to_csv("train_cleaned.csv")

1970 1970
